<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/3.%20Analizadores%20de%20salida%20(Output%20parsers).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. OutputParsers en Langchain**
---
Imagina que le preguntas a un LLM "¿Cuáles son los tres planetas más cercanos al sol?" y te responde: "Mercurio, Venus y la Tierra son los planetas más cercanos al sol". Si bien la respuesta es correcta para un humano, para que tu programa pueda usar esa información, lo ideal sería tenerla en un formato más manejable, como una lista o un objeto JSON. Aquí es donde entran en juego los Output Parsers.  

Los Output Parsers te permiten “forzar” o “guiar” al modelo para que devuelva la información según un formato deseado (por ejemplo, un JSON con campos específicos, una lista, etc).  

Convertir el texto libre, en información organizada, en algo que puedes usar directamente en tu código.



```
# Sin output parser: texto plano
"Tom Hanks ha actuado en Forrest Gump y Saving Private Ryan"

# Con output parser: estructura definida
{
  "actor": "Tom Hanks",
  "peliculas": ["Forrest Gump", "Saving Private Ryan"]
}
```

La estructuras de datos de salida son entre otras:

- Listas
- Enumeraciones
- Objetos JSON
- Diccionarios
- Modelos Pydantic


LangChain ofrece una variedad de Output Parsers preconstruidos para diferentes necesidades. Algunos de los mas usados son estos:

- **StrOutputParser:** Convierte la salida a string

- **CommaSeparatedListOutputParser**: Convierte la salida en una lista separada por comas. Útil para generar listas de elementos

- **EnumOutputParser**: Restringe la salida a un conjunto predefinido de valores. Perfecto para categorías o estados limitados. Idela cuedo se desea que el LLM elija de un conjunto de opciones.

- **JsonOutputParser** : Transforma la salida directamente en formato JSON. Ideal para respuestas estructuradas simples. Dos variantes principales:

    - SimpleJsonOutputParser
    - JsonOutputParser

- **DatetimeOutputParser**: Extrae y formatea fechas y horas. Útil para parsear información temporal

- **StructuredOutputParser**: Permite definir esquemas de salida más complejos. Configurable con múltiples campos

- **PydanticOutputParser**: Convierte la salida en objetos Pydantic. Permite definir estructuras de datos complejas. Gran flexibilidad para validación

- (**OutputFixingParser**: Intenta corregir salidas mal formateadas. Útil cuando el modelo no genera la estructura perfecta.)


Puedes ver todos los OutputParsers disponibles aquí:
https://python.langchain.com/docs/concepts/output_parsers/


# `with_structured_output`

Es un método nativo que aprovecha capacidades del modelo. Utiliza capacidades de llamada de función (function calling) del modelo. Es más eficiente y preciso. Esta soportado por modelos avanzados como OpenAI, Anthropic, Groq.

Lo trataremos al final de este cuaderno.

💡 **La mayor parte de modelos soportan esta función y es el futuro de la extracción estructurada en LangChain. Priorízalo cuando puedas.**

Puede consultarse una lista de modelos y sus capacidades aquí:
https://python.langchain.com/docs/integrations/chat/


Crear un ejemplo que dada una receta la presente estructurada en ingrdientes, pasos, etc
Esta en este video https://www.youtube.com/watch?v=lbWxastyWPw






#**2. Preparando el entorno del cuaderno**
----
Configuramos el entorno de trabajo para utilizar LangChain con distintos modelos de lenguaje (LLMs).

- Obtenemos las claves API para acceder a los servicios.

- Instalamos la librería LangChain y las integraciones necesarias para cada uno de estos proveedores.

- Importamos las clases específicas de LangChain que permiten crear plantillas de prompts e interactuar con los diferentes modelos de lenguaje, dejándolo todo listo para empezar a desarrollar aplicaciones basadas en LLMs. (Este codigo se explico con detalle en el primer cuaderno)

Comenta (#) las librerias y modelos que no desees usar.


In [1]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU

# Instalamos Rich para mejorar la salida
%pip install rich -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint

# Importamos las librerias para formatear mejor la salida
from IPython.display import Markdown, display
from rich import print as rprint

# **3. StrOutputParser**
---

`StrOutputParser` es el output parser más simple de LangChain. Su función principal es convertir la salida del modelo de lenguaje directamente en una cadena de texto sin realizar ninguna transformación estructural.

Características principales:
- Convierte la salida del modelo a texto plano
- No realiza ninguna validación o estructuración
- Útil cuando solo necesitas el texto sin procesar
- Muy ligero y directo

Casos de uso tipicos: Resumenes, traducciones simples, ...

## 🧪 Ejemplo: Generador de Resúmenes Ejecutivos

A partir de uos resultados de ventas (simulado) deseamos obtener un informe sobre el mismo.



In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Configuramos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY)

# Creamos un prompt para generar un resumen ejecutivo
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente experto en crear resúmenes ejecutivos concisos y claros."),
    ("human", "Genera un resumen ejecutivo sobre el siguiente informe de ventas: {informe}")
])

# Configuramos el output parser de tipo String
output_parser = StrOutputParser()

# Ejemplo de uso
informe_ventas = """
Ventas del Q1 2024:
- Ingresos totales: $1.5M
- Crecimiento interanual: 22%
- Producto más vendido: Software de gestión
- Principales mercados: Tecnología y Finanzas
"""

prompt = prompt_template.format_prompt(informe=informe_ventas)
respuesta = modelo.invoke(prompt).content
rprint(f"[bold]Respuesta del modelo:\n {respuesta}")
rprint("\n-----\n")
respuesta_formateada = output_parser.parse(respuesta)
rprint(f"[bold green4]Resumen generado:\n {respuesta_formateada}")


Respuesta del modelo:
 **Resumen Ejecutivo: Ventas Q1 2024**

**Resultados Clave:**

* Ingresos totales: $1.5M
* Crecimiento interanual: 22%

**Desempeño Destacado:**

* El Software de gestión fue el producto más vendido en el trimestre.
* Los sectores de Tecnología y Finanzas lideraron como nuestros principales mercados.

**Conclusión:**
Nuestro desempeño en el primer trimestre de 2024 muestra un crecimiento sólido, con un aumento del 22% en 
comparación con el año anterior. La demanda de nuestro Software de gestión y la participación en los sectores de 
Tecnología y Finanzas son factores clave en este éxito. Estos resultados nos posicionan bien para seguir creciendo 
y expandiendo nuestra presencia en el mercado en los próximos trimestres.

-----

Resumen generado:
 **Resumen Ejecutivo: Ventas Q1 2024**

**Resultados Clave:**

* Ingresos totales: $1.5M
* Crecimiento interanual: 22%

**Desempeño Destacado:**

* El Software de gestión fue el producto más vendido en el trimestre.
* Los sectores de Tecnología y Finanzas lideraron como nuestros principales mercados.

**Conclusión:**
Nuestro desempeño en el primer trimestre de 2024 muestra un crecimiento sólido, con un aumento del 22% en 
comparación con el año anterior. La demanda de nuestro Software de gestión y la participación en los sectores de 
Tecnología y Finanzas son factores clave en este éxito. Estos resultados nos posicionan bien para seguir creciendo 
y expandiendo nuestra presencia en el mercado en los próximos trimestres.

En cadenas muy simples donde solo esperas texto plano y no necesitas un control estricto sobre el tipo de dato, la diferencia práctica entre usar StrOutputParser explícitamente y no usar ningún OutputParser puede ser mínima. En muchos casos, obtendrás una salida de texto en ambos escenarios.

Sin embargo, usar StrOutputParser explícitamente es una buena práctica ya que mejora la claridad y legibilidad del código.
   
Ahora que entendemos cómo obtener texto plano, veamos cómo podemos empezar a estructurar la salida.

**Pero antes necesitamos conocer y manejar dos conceptos relacionados...**



# `Partial variables`

Las partial_variables son un mecanismo en LangChain para pre-rellenar variables en un prompt de manera parcial, antes de su uso final.  

Piénsalo de esta manera: un PromptTemplate es como una plantilla de texto con "huecos" que necesitas llenar para crear un prompt completo para el LLM. Hay dos formas principales de llenar estos huecos:

-   **input\_variables:** Estas son las variables que **cambian** cada vez que utilizas el prompt. Son los datos específicos que quieres que el LLM procese en cada llamada.
    
-   **partial\_variables:** Estas son las variables que tienen un valor **fijo** o **predefinido** para un uso particular del PromptTemplate. No cambian con cada llamada a la cadena o LLM que usa este prompt.

En este ejemplo nuestro PromptTemplate tiene 4 "huecos" o 4 variables. Pero al crearlo hemos precargado 3 de ellas con valores via partial variables. De esta al invocar el prompt (es un runnable !!) basta que pasemos una (tema).   
Sin imbargo podriamos pasar tambien las restanteas...

In [33]:
from langchain_core.prompts import PromptTemplate

# Prompt con partial_variables
prompt = PromptTemplate(
    template="Eres un {role} especializado en {area}. {instrucciones}",
    input_variables=["tema"],
    partial_variables={
        "role": "analista",
        "area": "tecnología",
        "instrucciones": "Proporciona un análisis detallado y objetivo."
    }
)

# Uso del prompt
resultado = prompt.invoke({"tema": "Inteligencia Artificial"})
print(f"{resultado}")

resultado = prompt.invoke({"tema": "Inteligencia Artificial", "instrucciones": "Contesta con un pareado que rime"})
rprint(f"[bold green4]{resultado}")


text='Eres un analista especializado en tecnología. Proporciona un análisis detallado y objetivo.'


text='Eres un analista especializado en tecnología. Contesta con un pareado que rime'

Un uso tipico de las partial_variables es usarlas para introducir en el prompt las instrucciones de l formato que proporciona Langchain

# `.get_format_instructions()`

**Obtener instrucciones de formato (opcional pero recomendado):** Muchos parsers tienen un método get\_format\_instructions() que devuelve texto que puedes incluir en tu prompt para guiar al LLM sobre el formato esperado.



In [5]:
from langchain.output_parsers import CommaSeparatedListOutputParser

# Crear el ListOutputParser
output_parser = CommaSeparatedListOutputParser()

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

Podriamos redactar nosotros mismos las instrucciones ? SI, sin duda. Esto es solo una funcion de utilidad que nos proporciona Langchain. Teoricamnte disponemos de esta forma de una redaccion tecnicamente correcta.

# **4. CommaSeparatedListOutputParser**
---

El `CommaSeparatedListOutputParser` en Langchain es un OutputParser **simple pero muy útil** diseñado para **interpretar la salida de un modelo de lenguaje (LLM) como una lista de elementos separados por comas.** Su función principal es tomar el texto generado por el LLM y **transformarlo en una lista de strings de Python**, donde cada string representa un elemento de la lista original que estaba separado por comas en el texto del LLM.

  

Imagina que le pides a un LLM que te dé "tres colores primarios separados por comas". Podrías esperar una respuesta como:

"rojo, azul, amarillo"

El CommaSeparatedListOutputParser toma esta cadena "rojo, azul, amarillo" y la procesa de la siguiente manera:

- **Divide la cadena:** Utiliza la coma (,) como delimitador para dividir la cadena en partes más pequeñas.
    
- **Elimina espacios en blanco (opcional):** Puede configurarse para eliminar espacios en blanco al principio y al final de cada parte extraída. Por defecto, suele hacerlo para limpiar la lista resultante.
    
- **Crea la lista:** Cada parte resultante se convierte en un elemento de una lista de Python.
    
-   **Casos de uso:** Obtener listas de elementos, como nombres, ideas, pasos a seguir, o categorías.
    
-   **Ventaja:** Simple y efectivo para extraer listas.




## 🧪 Ejemplo: Lista de ingredientes
Queremos obtener una lista de ingredientes para hacer una pizza casera y solo nos interesa la lista de ingredientes, pues la procesaremos posteriormente de alguan forma.

👀Observa el uso de las partial_variables paara introducir en el prompt las instrucciones de formato

👀Observa tambien el tipo de las dos respuestas. El primero es un string y no podriamos iterarlo. El segundo es una lista de python, si podemos iterarla.

In [34]:
from langchain.output_parsers import CommaSeparatedListOutputParser


# Crear el ListOutputParser
output_parser = CommaSeparatedListOutputParser()

# Crear el prompt
prompt_template = PromptTemplate(
    template="Genera una lista de ingredientes para hacer {receta} casera. Solo lista los ingredientes, uno por línea. Sin opciones\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)

# Generar la salida
prompt = prompt_template.format(receta="pizza")
respuesta = modelo.invoke(prompt).content

# Parsear la salida
respuesta_formateada = output_parser.parse(respuesta)

# Mostrar los resultados
rprint(f"Respuesta del modelo SIN FORMATEAR:\n [bold bright_cyan]{respuesta}")
rprint(type(respuesta))
rprint("\n\n")
rprint(f"Respuesta del modelo FORMATEADA:\n [bold spring_green3]{respuesta_formateada}")
rprint(type(respuesta_formateada))


Respuesta del modelo SIN FORMATEAR:
 Harina de trigo, levadura seca, sal, azúcar, agua tibia, aceite de oliva, salsa de tomate, queso mozzarella, queso
parmesano, orégano, pimiento, cebolla, champiñones, pepperoni

<class 'str'>

Respuesta del modelo FORMATEADA:
 ['Harina de trigo', 'levadura seca', 'sal', 'azúcar', 'agua tibia', 'aceite de oliva', 'salsa de tomate', 'queso 
mozzarella', 'queso parmesano', 'orégano', 'pimiento', 'cebolla', 'champiñones', 'pepperoni']

<class 'list'>

## 🧪 Ejemplo: Lista de etiquetas  
Deseamos que el modelo genere una lista de hastags (o etiquetas) a partir del tema de un articulo

In [35]:
from langchain.output_parsers import CommaSeparatedListOutputParser


# Crear el ListOutputParser
output_parser = CommaSeparatedListOutputParser()

# Crear el prompt
prompt_template = PromptTemplate(
    template="""Dame 5 etiquetas relevantes para un post de blog sobre: {tema}.
                Las etiquetas deben estar separadas por comas.""",
    input_variables=["tema"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)


# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)


# Formatear el prompt con el tema del blog
prompt = prompt_template.format(tema="Recetas de cocina vegana fáciles y rápidas para principiantes")

# Obtener la salida del LLM
respuesta = modelo.invoke(prompt).content

# Parsear la salida
respuesta_formateada = output_parser.parse(respuesta)

# Mostrar los resultados
rprint(f"Respuesta del modelo SIN FORMATEAR:\n [bold bright_cyan]{respuesta}")
rprint(type(respuesta))

rprint("\n\n")

rprint(f"Respuesta del modelo FORMATEADA:\n [bold spring_green3]{respuesta_formateada}")
rprint(type(respuesta_formateada))

rprint("\n\n")

# Imprimir las etiquetas generadas
print("Etiquetas sugeridas:")
for etiqueta in respuesta_formateada:
    rprint(f"[bold spring_green3] - {etiqueta.strip()}")


Respuesta del modelo SIN FORMATEAR:
 Recetas veganas, cocina vegana para principiantes, recetas fáciles y rápidas, comida vegana saludable, recetas 
veganas para empezar.

<class 'str'>

Respuesta del modelo FORMATEADA:
 ['Recetas veganas', 'cocina vegana para principiantes', 'recetas fáciles y rápidas', 'comida vegana saludable', 
'recetas veganas para empezar.']

<class 'list'>

Etiquetas sugeridas:


 - Recetas veganas

 - cocina vegana para principiantes

 - recetas fáciles y rápidas

 - comida vegana saludable

 - recetas veganas para empezar.

# **5. EnumOutputParser**
---

`EnumOutputParser` es un tipo de output parser en LangChain que se utiliza para restringir la salida de un modelo a un conjunto predefinido de valores. Esto es útil cuando se desea que la respuesta del modelo pertenezca a un conjunto específico de opciones, como categorías, estados o tipos.

### Características Principales:

-   **Restricción de Valores**: Permite definir un conjunto limitado de opciones que el modelo puede devolver.
-   **Validación Automática**: Si la salida del modelo no coincide con las opciones definidas, se puede manejar como un error.
-   **Facilita la Consistencia**: Asegura que las respuestas sean coherentes y dentro de un rango esperado.



## 🧪 Ejemplo de Caso de Uso: Clasificación de Sentimientos

Imaginemos que estamos construyendo un sistema que clasifica el sentimiento de comentarios de clientes sobre un producto. Queremos que el modelo devuelva solo tres categorías: "positivo", "negativo" y "neutral".

Este ejemplo que NO FUNCIONA muestra las limitaciones de los OutputParsers pero tambien unas de sus utilidades, poder hacer validaciones y atrapar el error.

In [36]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser

# Definimos las opciones de sentimiento
class Sentimientos(Enum):
    POSITIVO = "positivo"
    NEGATIVO = "negativo"
    NEUTRAL = "neutral"

# Crear el EnumOutputParser
output_parser = EnumOutputParser(enum=Sentimientos)

# Crear el prompt
prompt_template = PromptTemplate(
    template="Clasifica el siguiente comentario: {comentario}",
    input_variables=["comentario"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)


# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)


# Ejemplo de uso
comentario_cliente = "Me encanta este producto, es increíble y funciona muy bien."

prompt = prompt_template.format(comentario=comentario_cliente)

# Clasificamos el sentimiento
respuesta = modelo.invoke(prompt).content

# Parsear la salida
try:
    respuesta_formateada = output_parser.parse(respuesta)
except Exception as e:
    respuesta_formateada = "No se pudo clasificar el sentimiento"
    rprint(f"[bold white on red]Error al parsear la salida: {e}")


rprint("Sentimiento Clasificado:")
rprint(respuesta_formateada)


Error al parsear la salida: Response 'El comentario se puede clasificar como:

**Comentario positivo**

Razones:

* El usuario expresa su entusiasmo y satisfacción con el producto utilizando palabras como "Me encanta" y "es 
increíble".
* Menciona que el producto "funciona muy bien", lo que indica que ha tenido una experiencia exitosa con él.

En general, el tono del comentario es muy positivo y sugiere que el usuario está muy satisfecho con el producto.' 
is not one of the expected values: ['positivo', 'negativo', 'neutral']
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

Sentimiento Clasificado:

No se pudo clasificar el sentimiento

Observa que a pesar de que el modelo si evalua correctaement el sentimiento, NO DEVUELVE EXACTAMENTE lo que nececitamos. Y el OutputParser no consigue extraer de la resuesta el valor buscado.

Lanchain plantea esta opciones (en el enlace):

- Usar `with_structured_output`
- Usar LangGraph
- Mejorar el prompt
- Cambiar de modelo
- Usar reintentos (con algun parserfixing)

En este momento, la forma disponible para nosotros y ademas la mas simple y directa es simplemente mejorar el prompt

In [38]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser

# Definimos las opciones de sentimiento
class Sentimientos(Enum):
    POSITIVO = "positivo"
    NEGATIVO = "negativo"
    NEUTRAL = "neutral"

# Crear el EnumOutputParser
output_parser = EnumOutputParser(enum=Sentimientos)

# Crear el prompt MUCHO MAS PRECISO !!!
prompt_template = PromptTemplate(
    template="Clasifica el siguiente comentario: {comentario} Contesta solo en minusculas con 'positivo', 'negativo' o 'neutral y nada más",
    input_variables=["comentario"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)


# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)


# Ejemplo de uso
comentario_cliente = "Me encanta este producto, es increíble y funciona muy bien."

prompt = prompt_template.format(comentario=comentario_cliente)

# Clasificamos el sentimiento
respuesta = modelo.invoke(prompt).content

# Parsear la salida
try:
    respuesta_formateada = output_parser.parse(respuesta)
except Exception as e:
    respuesta_formateada = "No se pudo clasificar el sentimiento"
    rprint(f"[bold white on red]Error al parsear la salida: {e}")


rprint("Sentimiento Clasificado:")
rprint(respuesta_formateada)
rprint(type(respuesta_formateada))


Sentimiento Clasificado:

Sentimientos.POSITIVO

<enum 'Sentimientos'>

¿ Que pinta aqui pues el OutputParser ? Esto lo podriamos simplemente con el prompt. SI.  

El OutputParser nos proporciona validacion de datos, que es importante sobre todo en el momento del desarrolo y tipo de datos. El resultado NO es un string, sino algo del tipo Enum que hemos definido que por ejemplo podemos iterar.

https://rico-schmidt.name/pymotw-3/enum/index.html

# **6. SimpleJsonOutputParser**
---

El SimpleJsonOutputParser es una herramienta de LangChain que se utiliza para convertir el texto de salida de un modelo de lenguaje en un objeto JSON estructurado. Es particularmente útil cuando necesitas obtener datos estructurados de tus LLMs (Large Language Models).

### Funcionalidad principal

- Convierte respuestas de texto en formato JSON
- Maneja errores de parseo
- Es sencillo de implementar en tu flujo de trabajo con LLMs

## 🧪 Ejemplo : Queremos obtener cierta informacion de una ciudad para procesarla posteriormente

In [39]:
from langchain_core.output_parsers import SimpleJsonOutputParser


# Crear el parser
parser = SimpleJsonOutputParser()

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)

# Hacer una consulta que espera una respuesta estructurada
# Importante aqui las dobles llaves para que no se interprete como variables
template = """
Proporciona información sobre {ciudad} con el siguiente formato:

{{
  "ciudad": "nombre de la ciudad",
  "pais": "país donde se encuentra",
  "poblacion": "número aproximado de habitantes",
  "atracciones": ["lista", "de", "atracciones", "principales"]
}}
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["ciudad"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

prompt = prompt_template.format(ciudad="Barcelona")

# Obtener la respuesta del modelo
respuesta = modelo.invoke(prompt).content

# Parsear la respuesta a JSON
resultado = parser.parse(respuesta)

# Ahora resultado es un diccionario Python
rprint(resultado)
rprint(type(resultado))  # <class 'dict'>
rprint(f"Primera atracción: {resultado['atracciones'][0]}")


{
    'ciudad': 'Barcelona',
    'pais': 'España',
    'poblacion': '1,6 millones',
    'atracciones': [
        'La Sagrada Familia',
        'Park Güell',
        'La Rambla',
        'Barceloneta',
        'Camp Nou',
        'Museo Picasso',
        'Casa Batlló',
        'La Boqueria'
    ]
}

<class 'dict'>

Primera atracción: La Sagrada Familia

## 🧪 Ejemplo: Analisis de sentimientos (un poco más elaborado)

👀 No vamos a usar PromptTempplate y directamente nos las arreglamos con f-strings

👀 El modelo devuleve la chachara habitual junto con el formato JSON solicitado. El parser es capaz de extraer de todo ello la infomacion JSON qu no interesa en el formato adecuado

In [41]:
from langchain_core.output_parsers import SimpleJsonOutputParser

# Crear el parser
parser = SimpleJsonOutputParser()

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)


# Comentario de un cliente sobre un producto
comentario_cliente = """
Compré este teléfono hace un mes y estoy muy contento con la calidad de la cámara y la batería dura todo el día.
Sin embargo, el software tiene algunos fallos y a veces se congela cuando uso múltiples aplicaciones al mismo tiempo.
"""

# Prompt para analizar el sentimiento y extraer insights
prompt = f"""
Analiza el siguiente comentario de un cliente y devuelve la información en formato JSON:

Comentario: {comentario_cliente}

El JSON debe tener esta estructura:
{{
  "sentimiento_general": "positivo/negativo/neutral",
  "puntuacion": "valor numérico de 1 a 10",
  "aspectos_positivos": ["lista", "de", "aspectos", "positivos"],
  "aspectos_negativos": ["lista", "de", "aspectos", "negativos"],
  "recomendaciones": ["lista", "de", "posibles", "mejoras"]
}}
"""

# Obtener respuesta y parsear
respuesta = modelo.invoke(prompt).content
respuestaf = parser.parse(respuesta)


rprint(respuesta)
rprint(type(respuesta))
rprint("\n\n")
rprint(respuestaf)
rprint(type(respuestaf))
rprint("\n\n")

# Usar los datos estructurados
rprint(f"Sentimiento: {respuestaf['sentimiento_general']}")
rprint(f"Puntuación: {respuestaf['puntuacion']}/10")
rprint("Aspectos positivos:", ", ".join(respuestaf["aspectos_positivos"]))
rprint("Aspectos negativos:", ", ".join(respuestaf["aspectos_negativos"]))

Análisis del comentario del cliente:

El comentario del cliente expresa una opinión generalmente positiva sobre el teléfono, destacando la calidad de la 
cámara y la duración de la batería. Sin embargo, también menciona algunos fallos en el software y problemas de 
congelamiento cuando se utilizan múltiples aplicaciones al mismo tiempo.

A continuación, se presenta la información en formato JSON:

```json
{
  "sentimiento_general": "positivo",
  "puntuacion": 8,
  "aspectos_positivos": [
    "Calidad de la cámara",
    "Duración de la batería"
  ],
  "aspectos_negativos": [
    "Fallos en el software",
    "Problemas de congelamiento con múltiples aplicaciones"
  ],
  "recomendaciones": [
    "Mejorar la estabilidad del software",
    "Optimizar el rendimiento para evitar congelamientos",
    "Realizar actualizaciones regulares para solucionar fallos"
  ]
}
```

En este análisis, se ha considerado que el sentimiento general es positivo debido a que el cliente está "muy 
contento" con algunos aspectos del teléfono, aunque también menciona algunos problemas. La puntuación se ha 
establecido en 8, considerando que el cliente está satisfecho con la mayoría de los aspectos del teléfono, pero no 
completamente. Los aspectos positivos y negativos se han identificado según el comentario del cliente, y las 
recomendaciones se han sugerido para abordar los problemas mencionados.

<class 'str'>

{
    'sentimiento_general': 'positivo',
    'puntuacion': 8,
    'aspectos_positivos': ['Calidad de la cámara', 'Duración de la batería'],
    'aspectos_negativos': ['Fallos en el software', 'Problemas de congelamiento con múltiples aplicaciones'],
    'recomendaciones': [
        'Mejorar la estabilidad del software',
        'Optimizar el rendimiento para evitar congelamientos',
        'Realizar actualizaciones regulares para solucionar fallos'
    ]
}

<class 'dict'>

Sentimiento: positivo

Puntuación: 8/10

Aspectos positivos: Calidad de la cámara, Duración de la batería

Aspectos negativos: Fallos en el software, Problemas de congelamiento con múltiples aplicaciones

# **7. JsonOutputParser**
---

JsonOutputParser es otro OutputParser dissponible en el framewwork LangChain.

## Principales diferencias

1. **Complejidad y flexibilidad**:
    - **SimpleJsonOutputParser**: Como su nombre indica, es más simple. Toma una cadena de texto que contiene JSON válido y la convierte en un objeto Python.
    - **JsonOutputParser**: Es más complejo y flexible, permitiendo definir un esquema específico para la estructura JSON esperada.
2. **Esquemas y validación**:
    - **SimpleJsonOutputParser**: No requiere definir un esquema previo; simplemente intenta parsear cualquier JSON válido.
    - **JsonOutputParser**: Requiere definir la estructura esperada, lo que proporciona validación y guía al LLM sobre cómo estructurar su respuesta.
3. **Instrucciones al modelo**:
    - **SimpleJsonOutputParser**: No genera instrucciones específicas para el modelo.
    - **JsonOutputParser**: Proporciona instrucciones detalladas al modelo sobre el formato requerido con `get_format_instructions()`.
4. **Manejo de errores**:
    - **SimpleJsonOutputParser**: Manejo básico de errores de parseo.
    - **JsonOutputParser**: Manejo más robusto de errores con validación contra el esquema definido.

## ¿Cuándo usar cada uno?

**Usa SimpleJsonOutputParser cuando:**

- Necesitas una solución rápida y sencilla
- La estructura JSON puede variar o no es crítica
- No requieres validación estricta del esquema

**Usa JsonOutputParser cuando:**

- Necesitas validar contra un esquema específico
- Quieres proporcionar instrucciones detalladas al modelo
- La estructura de datos es compleja o crítica para tu aplicación
- Trabajas con tipos de datos específicos que requieren validación

El JsonOutputParser es especialmente útil en escenarios empresariales donde la consistencia y validación de los datos son cruciales, como en análisis de productos, extracción de información de documentos, o procesamiento de datos estructurados desde texto libre.

## 🧪 Ejemplo: Deseamos obtener la bibliografia de un autor

👀 Observa lo importante que es el prompt para conseguir que el modelo cree una respuesta que pueda ser parseada por el OutputParser.

El OutputParser ademas de validacion de datos nos propociona los datos en el tipo deseado y no una simple repesentacion en string. **Observa los tipos de de respuesta y respuesta_formateada**

In [42]:
from pydantic import BaseModel, Field
from typing import List
from langchain_core.output_parsers import JsonOutputParser

# Definir la estructura para un libro
class Libro(BaseModel):
    titulo: str = Field(description="Título del libro")
    año: int = Field(description="Año de publicación")

# Definir la estructura para la bibliografía completa
class Bibliografia(BaseModel):
    bibliografia: List[Libro] = Field(description="Lista de libros publicados por el autor")

# Crear el JsonOutputParser
output_parser = JsonOutputParser(pydantic_model=Bibliografia)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()

# Crear el prompt
texto_prompt = """
Proporciona ÚNICAMENTE la bibliografía del autor: {autor}.

INSTRUCCIONES ESTRICTAS:
1. Sigue EXACTAMENTE el formato JSON especificado a continuación.
2. NO incluyas campos adicionales como nacionalidad, nacimiento, fallecimiento, premios, etc.
3. Solo crea una lista de sus libros con título y año.
4. La lista de libros debe usar la clave "bibliografia", no "obras" ni ninguna otra.
5. Cada libro debe tener SOLO los campos "titulo" y "año".

Es CRÍTICO seguir este formato exacto:

{format_instructions}

IMPORTANTE: No agregues ningún campo que no esté especificado en el esquema. Tu respuesta debe ser únicamente un JSON válido con la estructura exacta solicitada.
"""

prompt_template = PromptTemplate(
    template=texto_prompt,
    input_variables=["autor"],
    partial_variables={"format_instructions": format_instructions}
)

# Generar el prompt para el LLM
autor = "Gabriel García Márquez"
prompt = prompt_template.format(autor=autor)

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)

# Hacemos la llamada
respuesta = modelo.invoke(prompt).content

# Analizamos la respuesta
respuesta_formateada = output_parser.parse(respuesta)

rprint(respuesta)
rprint(type(respuesta))

rprint(respuesta_formateada)
rprint(type(respuesta_formateada))

# Analizamos la respuesta
try:
    respuesta_formateada = output_parser.parse(respuesta)

    # Imprimimos los resultados
    print(f"Autor: {autor}")
    print("Bibliografía:")
    for libro in respuesta_formateada["bibliografia"]:
        print(f"Año: {libro['año']}, Título: {libro['titulo']}")
except Exception as e:
    print(f"Error al parsear la respuesta: {e}")
    print("La respuesta del modelo no cumple con el esquema esperado.")

```json
{
  "bibliografia": [
    {
      "titulo": "La hojarasca",
      "año": 1955
    },
    {
      "titulo": "El coronel no tiene quien le escriba",
      "año": 1961
    },
    {
      "titulo": "Los funerales de la Mamá Grande",
      "año": 1962
    },
    {
      "titulo": "Cien años de soledad",
      "año": 1967
    },
    {
      "titulo": "Relato de un náufrago",
      "año": 1970
    },
    {
      "titulo": "El ahogado más hermoso del mundo",
      "año": 1970
    },
    {
      "titulo": "Crónica de una muerte anunciada",
      "año": 1981
    },
    {
      "titulo": "El amor en los tiempos del cólera",
      "año": 1985
    },
    {
      "titulo": "Doce cuentos peregrinos",
      "año": 1992
    },
    {
      "titulo": "Del amor y otros demonios",
      "año": 1994
    },
    {
      "titulo": "Noticia de un secuestro",
      "año": 1996
    },
    {
      "titulo": "Vivir para contarla",
      "año": 2002
    }
  ]
}
```

<class 'str'>

{
    'bibliografia': [
        {'titulo': 'La hojarasca', 'año': 1955},
        {'titulo': 'El coronel no tiene quien le escriba', 'año': 1961},
        {'titulo': 'Los funerales de la Mamá Grande', 'año': 1962},
        {'titulo': 'Cien años de soledad', 'año': 1967},
        {'titulo': 'Relato de un náufrago', 'año': 1970},
        {'titulo': 'El ahogado más hermoso del mundo', 'año': 1970},
        {'titulo': 'Crónica de una muerte anunciada', 'año': 1981},
        {'titulo': 'El amor en los tiempos del cólera', 'año': 1985},
        {'titulo': 'Doce cuentos peregrinos', 'año': 1992},
        {'titulo': 'Del amor y otros demonios', 'año': 1994},
        {'titulo': 'Noticia de un secuestro', 'año': 1996},
        {'titulo': 'Vivir para contarla', 'año': 2002}
    ]
}

<class 'dict'>

Autor: Gabriel García Márquez
Bibliografía:
Año: 1955, Título: La hojarasca
Año: 1961, Título: El coronel no tiene quien le escriba
Año: 1962, Título: Los funerales de la Mamá Grande
Año: 1967, Título: Cien años de soledad
Año: 1970, Título: Relato de un náufrago
Año: 1970, Título: El ahogado más hermoso del mundo
Año: 1981, Título: Crónica de una muerte anunciada
Año: 1985, Título: El amor en los tiempos del cólera
Año: 1992, Título: Doce cuentos peregrinos
Año: 1994, Título: Del amor y otros demonios
Año: 1996, Título: Noticia de un secuestro
Año: 2002, Título: Vivir para contarla


Observa que el tipo de la respuesta obtenida es un diccionario de Python

## **🧪 Ejemplo: Analisis de producto detallado**

Dada un descripcion textual de un articulo, deseamos obtener un analisis detallado del producto con cierta estructura de datos


In [44]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from typing import List


# Definir el esquema que esperamos recibir
class ProductoAnalisis(BaseModel):
    nombre: str = Field(description="Nombre del producto analizado")
    categoria: str = Field(description="Categoría principal del producto")
    ventajas: List[str] = Field(description="Lista de puntos fuertes o ventajas del producto")
    desventajas: List[str] = Field(description="Lista de puntos débiles o desventajas del producto")
    puntuacion: int = Field(description="Puntuación de 1 a 10")
    recomendado: bool = Field(description="Si el producto es recomendable")
    mejor_para: List[str] = Field(description="Tipos de usuarios para los que este producto es más adecuado")

# Crear el parser con nuestro esquema
parser = JsonOutputParser(pydantic_model=ProductoAnalisis)

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)

# Crear el template con las instrucciones de formato
template = """
Analiza el siguiente producto y proporciona un análisis detallado:

Producto: {producto}

Es CRÍTICO seguir este formato exacto:

{format_instructions}

Debes realizar un análisis del producto indicado. Presenta los resultados estrictamente siguiendo este esquema:

nombre: (Nombre exacto del producto)

categoria: (Categoría principal a la que pertenece el producto)

ventajas:

(Lista claramente identificada de puntos fuertes o ventajas)

desventajas:

(Lista claramente identificada de puntos débiles o desventajas)

puntuacion: (Valoración numérica del producto entre 1 y 10, donde 10 es excelente)

recomendado: (Indica explícitamente "Sí" o "No" dependiendo de si recomiendas el producto)

mejor_para:

(Lista concreta de perfiles o tipos de usuarios para los que este producto sería más adecuado)

Asegúrate de completar cada sección de manera detallada y precisa, sin omitir ningún campo.

IMPORTANTE: No agregues ningún campo que no esté especificado en el esquema. Tu respuesta debe ser únicamente un JSON válido con la estructura exacta solicitada.


"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["producto"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# Producto a analizar
descripcion_producto = """
Auriculares inalámbricos XSound Pro - Con cancelación activa de ruido,
30 horas de batería, conexión Bluetooth 5.2, resistencia al agua IPX4,
y sistema de micrófono dual para llamadas. Precio: 129,99€
"""

prompt = prompt_template.format(producto=descripcion_producto)


# Obtener y parsear la respuesta
respuesta = modelo.invoke(prompt)
respuestaf= parser.parse(respuesta.content)


# Impresión estructurada
print(f"Nombre: {respuestaf['nombre']}")
print(f"Categoría: {respuestaf['categoria']}\n")

print("Ventajas:")
for ventaja in respuestaf['ventajas']:
    print(f"- {ventaja}")

print("\nDesventajas:")
for desventaja in respuestaf['desventajas']:
    print(f"- {desventaja}")

print(f"\nPuntuación: {respuestaf['puntuacion']}/10")
print(f"Recomendado: {respuestaf['recomendado']}")

print("\nMejor para:")
for usuario in respuestaf['mejor_para']:
    print(f"- {usuario}")

Nombre: Auriculares inalámbricos XSound Pro
Categoría: Electrónica de audio

Ventajas:
- Cancelación activa de ruido para una mejor experiencia de escucha
- 30 horas de batería para un uso prolongado
- Conexión Bluetooth 5.2 para una conexión estable y rápida
- Resistencia al agua IPX4 para protección contra el sudor y la lluvia
- Sistema de micrófono dual para llamadas claras y nítidas

Desventajas:
- Precio puede ser considerado alto para algunos usuarios
- No se menciona la calidad del sonido en términos de frecuencia y respuesta
- No se proporciona información sobre la compatibilidad con dispositivos específicos

Puntuación: 8/10
Recomendado: Sí

Mejor para:
- Deportistas y personas que practican ejercicio regularmente
- Usuarios que buscan una buena calidad de sonido y cancelación de ruido
- Personas que necesitan un producto resistente al agua y duradero
- Usuarios que valoran la comodidad y la facilidad de uso de un producto inalámbrico


# **8. StructuredOutputParser**
---
StructuredOutputParser es un parser de salida estructurado pero sencillo pero a veces no necesitamos mucho mas.

- **Propósito**: Generar respuestas estructuradas en formatos simples, ideal para modelos pequeños o con restricciones.
    
- **Características clave**:  
    • **Siempre devuelve un diccionario con campos de tipo _string_.**  
    • Permite definir una estructura de salida predeterminada.  
    • Menos complejo que opciones como _PydanticOutputParser_ (no soporta datos complejos).
    
- **Limitaciones**:  
    • **Solo admite campos de texto (_string_).**  
    • Menos flexible para escenarios que requieren tipos de datos avanzados.
    
- **Casos de uso**:  
    • Proyectos con requisitos de estructura básicos.  
    • Compatibilidad con modelos de lenguaje menos potentes o entornos limitados.

** Realmente puede manejar listas sencillas, pero es mejor evitarlo. Si debe haber listas en la respuesta, usa algun JSON *texto en cursiva*

## **🧪 Ejemplo: Crear datos dummy de usuarios**
Observa que por las limitaciones de este pareser, que solo genera strings la edad sera un string que habra que convertir.

In [47]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

#Creamos un esquema de respuesta (ResponseSchema) para cada campo que queremos extraer:
esquema_respuesta = [
    ResponseSchema(name="nombre", description="El nombre del usuario"),
    ResponseSchema(name="edad", description="La edad del usuario"),
    ResponseSchema(name="email", description="El correo electrónico del usuario")
]

# Crear el StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(esquema_respuesta)
format_instructions = output_parser.get_format_instructions()

# Crear el prompt
prompt_template = PromptTemplate(
    template="Genera información de un usuario ficticio.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": format_instructions}
)

# Generar el prompt para el LLM
prompt = prompt_template.format()


# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=1)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=1)

# Hacemos la llamada
respuesta = modelo.invoke(prompt).content
respuesta_formateada = output_parser.parse(respuesta)

rprint(respuesta)
rprint(type(respuesta))
rprint("\n")
rprint(respuesta_formateada)
rprint(type(respuesta_formateada))
rprint("\n")


# Usamos los datos estructurados
rprint(f"Nombre: {respuesta_formateada['nombre']}")
rprint(f"Edad: {int(respuesta_formateada['edad'])}")
rprint(f"Email: {respuesta_formateada['email']}")


```json
{
        "nombre": "Elena García",
        "edad": "32",
        "email": "elenagarcia@example.com"
}
```

<class 'str'>

{'nombre': 'Elena García', 'edad': '32', 'email': 'elenagarcia@example.com'}

<class 'dict'>

Nombre: Elena García

Edad: 32

Email: elenagarcia@example.com

## **🧪 Ejemplo: Creacion de un test**
---

Vamos a diseñar una consulta para un modelo de lenguaje (LLM) que sirva como base para generar un test de preguntas sobre un tema específico. Proporcionaremos el tema y un nivel de dificultad (bajo, medio, alto), y el LLM deberá generar el texto de la pregunta, tres opciones de respuesta y el índice de la respuesta correcta. La respuesta debe estar estructurada en un diccionario con el siguiente formato:

```
{
    "pregunta": "Texto de la pregunta",
    "opciones": ["Opción 1", "Opción 2", "Opción 3"],
    "respuesta_correcta": índice_de_la_opción_correcta
}
```

Este formato permitirá una fácil interpretación y uso de la pregunta generada.

Como no estamos seguros de la capcidad de este OutputParser de generar listas, las opciones seran tres campos string independientes.


In [48]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

#Creamos un esquema de respuesta (ResponseSchema) para cada campo que queremos extraer:
esquema_respuesta = [
            ResponseSchema(name="pregunta", description="Texto de la pregunta generada."),
            ResponseSchema(name="opcion1", description="Primera opción de respuesta."),
            ResponseSchema(name="opcion2", description="Segunda opción de respuesta."),
            ResponseSchema(name="opcion3", description="Tercera opción de respuesta."),
            ResponseSchema(name="respuesta_correcta", description="Índice de la opción correcta (1, 2 o 3).")
                    ]

# Crear el StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(esquema_respuesta)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

# Crear el prompt
plantilla = """
        Genera una pregunta de test sobre el tema: {tema}.
        El nivel de dificultad debe ser: {nivel}.
        Genera tambien tres posibles respuestas (opcion1, opcion2, opcion3)
        Una de ellas debe ser la correcta
        Indica el numero (1,2,3) de la respuesta correcta (respuesta_correcta)


        {format_instructions}
        """

prompt_template = PromptTemplate(
    template=plantilla,
    input_variables=["tema", "nivel"],
    partial_variables={"format_instructions": format_instructions}
)

# Generar el prompt para el LLM
prompt = prompt_template.format(tema="LangChain", nivel="medio")
print("Prompt generado:\n", prompt)

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)

# Hacemos la llamada
respuesta = modelo.invoke(prompt).content
respuesta_formateada = output_parser.parse(respuesta)

rprint(respuesta)
rprint(type(respuesta))

rprint(respuesta_formateada)
rprint(type(respuesta_formateada))

# Usamos los datos estructurados
rprint(f"Pregunta: {respuesta_formateada['pregunta']}")
rprint(f"Opción 1: {respuesta_formateada['opcion1']}")
rprint(f"Opción 2: {respuesta_formateada['opcion2']}")
rprint(f"Opción 3: {respuesta_formateada['opcion3']}")
rprint(f"Respuesta correcta: Opción {respuesta_formateada['respuesta_correcta']}")

Prompt generado:
 
        Genera una pregunta de test sobre el tema: LangChain.
        El nivel de dificultad debe ser: medio.
        Genera tambien tres posibles respuestas (opcion1, opcion2, opcion3)
        Una de ellas debe ser la correcta
        Indica el numero (1,2,3) de la respuesta correcta (respuesta_correcta)


        The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"pregunta": string  // Texto de la pregunta generada.
	"opcion1": string  // Primera opción de respuesta.
	"opcion2": string  // Segunda opción de respuesta.
	"opcion3": string  // Tercera opción de respuesta.
	"respuesta_correcta": string  // Índice de la opción correcta (1, 2 o 3).
}
```
        


```json
{
        "pregunta": "¿Cuál es el propósito principal de LangChain en el contexto de la inteligencia artificial y el
procesamiento del lenguaje natural?",
        "opcion1": "Desarrollar modelos de aprendizaje automático para tareas de visión por computadora",
        "opcion2": "Crear una plataforma para la integración y el desarrollo de aplicaciones basadas en lenguaje 
natural",
        "opcion3": "Diseñar algoritmos para la compresión de datos en redes neuronales",
        "respuesta_correcta": "2"
}
```

<class 'str'>

{
    'pregunta': '¿Cuál es el propósito principal de LangChain en el contexto de la inteligencia artificial y el 
procesamiento del lenguaje natural?',
    'opcion1': 'Desarrollar modelos de aprendizaje automático para tareas de visión por computadora',
    'opcion2': 'Crear una plataforma para la integración y el desarrollo de aplicaciones basadas en lenguaje 
natural',
    'opcion3': 'Diseñar algoritmos para la compresión de datos en redes neuronales',
    'respuesta_correcta': '2'
}

<class 'dict'>

Pregunta: ¿Cuál es el propósito principal de LangChain en el contexto de la inteligencia artificial y el 
procesamiento del lenguaje natural?

Opción 1: Desarrollar modelos de aprendizaje automático para tareas de visión por computadora

Opción 2: Crear una plataforma para la integración y el desarrollo de aplicaciones basadas en lenguaje natural

Opción 3: Diseñar algoritmos para la compresión de datos en redes neuronales

Respuesta correcta: Opción 2

## **🧪 Ejemplo: Creacion de un test II**
---
Vamos a comprobar que el StructuredOutputParser puede gestionar listas sencillas con exito.  
👀 **Observa como el diccionario devuelto por el parser, contiene una lista de strings con las preguntas**


In [50]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Creamos un esquema de respuesta (ResponseSchema) para cada campo que queremos extraer:
# Pero ahora usamos una lista, al menos asi lo indicamos en el texto de la descripcion
response_schemas = [
            ResponseSchema(name="pregunta", description="Texto de la pregunta generada."),
            ResponseSchema(name="opciones", description="LISTA de tres opciones de respuesta."),
            ResponseSchema(name="respuesta_correcta", description="Índice de la opción correcta (1, 2 o 3).")
                    ]

# Crear el StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

# Crear el prompt
texto_prompt = """
        Genera una pregunta de test sobre el tema: {tema}.
        El nivel de dificultad debe ser: {nivel}.
        La pregunta debe tener tres opciones de respuesta y una respuesta correcta.

        {format_instructions}
        """

prompt_template = PromptTemplate(
    template=texto_prompt,
    input_variables=["tema", "nivel"],
    partial_variables={"format_instructions": format_instructions}
)

# Generar el prompt para el LLM
prompt = prompt_template.format(tema="LangChain", nivel="medio")
print("Prompt generado:\n", prompt)

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)

# Hacemos la llamada
respuesta = modelo.invoke(prompt).content
respuesta_formateada = output_parser.parse(respuesta)

rprint(respuesta)
rprint(type(respuesta))

rprint(respuesta_formateada)
rprint(type(respuesta_formateada))

# Usamos los datos estructurados
rprint(f"Pregunta: {respuesta_formateada['pregunta']}")
rprint("Opciones:")
for i, opcion in enumerate(respuesta_formateada['opciones'], start=1):
    rprint(f"{opcion}")
rprint(f"Respuesta correcta: Opción {respuesta_formateada['respuesta_correcta']}")

Prompt generado:
 
        Genera una pregunta de test sobre el tema: LangChain.
        El nivel de dificultad debe ser: medio.
        La pregunta debe tener tres opciones de respuesta y una respuesta correcta.

        The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"pregunta": string  // Texto de la pregunta generada.
	"opciones": string  // LISTA de tres opciones de respuesta.
	"respuesta_correcta": string  // Índice de la opción correcta (1, 2 o 3).
}
```
        


```json
{
        "pregunta": "¿Cuál es el propósito principal de LangChain en el contexto de la inteligencia artificial y el
procesamiento del lenguaje natural?",
        "opciones": [
                "Desarrollar modelos de lenguaje para tareas específicas",
                "Crear una plataforma para la integración de modelos de lenguaje y aplicaciones",
                "Mejorar la comprensión del lenguaje humano a través de la investigación"
        ],
        "respuesta_correcta": 2
}
```

<class 'str'>

{
    'pregunta': '¿Cuál es el propósito principal de LangChain en el contexto de la inteligencia artificial y el 
procesamiento del lenguaje natural?',
    'opciones': [
        'Desarrollar modelos de lenguaje para tareas específicas',
        'Crear una plataforma para la integración de modelos de lenguaje y aplicaciones',
        'Mejorar la comprensión del lenguaje humano a través de la investigación'
    ],
    'respuesta_correcta': 2
}

<class 'dict'>

...

Opciones:

Desarrollar modelos de lenguaje para tareas específicas

Crear una plataforma para la integración de modelos de lenguaje y aplicaciones

Mejorar la comprensión del lenguaje humano a través de la investigación

Respuesta correcta: Opción 2

## **🧪 Ejemplo: Extracción de datos en anuncios inmobiliarios**

Disponemos del texto de un anuncio inmobiliario y desamos extraer la informacion de forma estructurada.

In [52]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema


# Definir los esquemas para la extracción de datos inmobiliarios
response_schemas = [
    ResponseSchema(name="tipo_propiedad", description="Tipo de propiedad (apartamento, casa, chalet, etc.)"),
    ResponseSchema(name="precio", description="Precio de la propiedad en formato numérico sin símbolos"),
    ResponseSchema(name="moneda", description="Moneda del precio (EUR, USD, etc.)"),
    ResponseSchema(name="superficie", description="Superficie en metros cuadrados, solo número"),
    ResponseSchema(name="habitaciones", description="Número de habitaciones, solo número"),
    ResponseSchema(name="baños", description="Número de baños, solo número"),
    ResponseSchema(name="ubicacion", description="Ubicación de la propiedad (barrio, ciudad)"),
    ResponseSchema(name="caracteristicas", description="Lista de características destacadas de la propiedad"),
    ResponseSchema(name="estado", description="Estado de la propiedad (nuevo, reformado, a reformar, etc.)"),
]

# Crear el parser
parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Obtener instrucciones de formato
format_instructions = parser.get_format_instructions()

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)

# Texto del anuncio inmobiliario
descripcion_inmueble = """
Magnífico piso de 95 m² en el corazón de Salamanca. Consta de 3 dormitorios, 2 baños completos,
cocina equipada y amplio salón con balcón. La propiedad ha sido recientemente reformada con
materiales de alta calidad. Dispone de calefacción central, aire acondicionado, suelos de parquet,
armarios empotrados y plaza de garaje incluida. Edificio con ascensor y servicio de portería.
Excelente ubicación cerca de todos los servicios, comercios y transporte público.
Precio: 450.000€. Gastos de comunidad: 150€/mes.
"""

# Crear el prompt
prompt = f"""
Extrae la información clave del siguiente anuncio inmobiliario:

{descripcion_inmueble}

{format_instructions}
"""

# Obtener la respuesta
respuesta = modelo.invoke(prompt).content
print(respuesta)

# Parsear la respuesta
propiedades = parser.parse(respuesta)

rprint(propiedades)
rprint(type(propiedades))

# Imprimir los resultados de forma estructurada
rprint(f"[bold spring_green3]Tipo: {propiedades['tipo_propiedad']}")
rprint(f"[bold spring_green3]Precio: {propiedades['precio']} {propiedades['moneda']}")
rprint(f"[bold spring_green3]Superficie: {propiedades['superficie']} m²")
rprint(f"[bold spring_green3]Habitaciones: {propiedades['habitaciones']}")
rprint(f"[bold spring_green3]Baños: {propiedades['baños']}")
rprint(f"[bold spring_green3]Ubicación: {propiedades['ubicacion']}")
rprint(f"[bold spring_green3]Estado: {propiedades['estado']}")
rprint(f"[bold spring_green3]Caracteristicas: {propiedades['caracteristicas']}")


```json
{
	"tipo_propiedad": "piso",
	"precio": "450000",
	"moneda": "EUR",
	"superficie": "95",
	"habitaciones": "3",
	"baños": "2",
	"ubicacion": "Salamanca",
	"caracteristicas": "calefacción central, aire acondicionado, suelos de parquet, armarios empotrados, plaza de garaje incluida, ascensor, servicio de portería",
	"estado": "reformado"
}
```


{
    'tipo_propiedad': 'piso',
    'precio': '450000',
    'moneda': 'EUR',
    'superficie': '95',
    'habitaciones': '3',
    'baños': '2',
    'ubicacion': 'Salamanca',
    'caracteristicas': 'calefacción central, aire acondicionado, suelos de parquet, armarios empotrados, plaza de 
garaje incluida, ascensor, servicio de portería',
    'estado': 'reformado'
}

<class 'dict'>

Tipo: piso

Precio: 450000 EUR

...

Habitaciones: 3

Baños: 2

Ubicación: Salamanca

Estado: reformado

Caracteristicas: calefacción central, aire acondicionado, suelos de parquet, armarios empotrados, plaza de garaje 
incluida, ascensor, servicio de portería

# **9. PydanticOutputParser**
---
Convierte la salida a un modelo de datos Pydantic.
Pydantic es una poderosa biblioteca en Python diseñada para la validación y gestión de datos. Es especialmente útil cuando trabajas con datos que necesitan ser validados y transformados, como datos de entrada de API o formularios. Esta incluida como una dependencia en Langchain.

El `PydanticOutputParser` es una herramienta avanzada de LangChain que combina el poder de la validación de Pydantic con la capacidad de estructurar las salidas de los modelos de lenguaje. A diferencia del `SimpleJsonOutputParser` o el `StructuredOutputParser`, este parser utiliza modelos Pydantic completos para definir esquemas de datos rigurosos.

## Características principales

- Utiliza modelos Pydantic para definir la estructura de datos esperada
- Proporciona validación de tipos robusta
- Soporta esquemas de datos complejos y anidados
- Genera instrucciones detalladas para guiar al modelo de lenguaje
- Convierte automáticamente la respuesta en una instancia del modelo Pydantic

## Funcionamiento

1. Se define un modelo Pydantic que represente la estructura de datos deseada
2. Se crea un `PydanticOutputParser` basado en ese modelo
3. Se generan instrucciones para el modelo de lenguaje
4. Se parsea la respuesta para obtener una instancia del modelo Pydantic

Si buscas simplicidad y rapidez, StructuredOutputParser es una buena opción pero si necesitas validación de datos robusta y estás utilizando Pydantic en tu proyecto, PydanticOutputParser es la mejor alternativa.



## **🧪 Ejemplo: Creacion de un test III**

Vamos a crear de nuevo el ejemplo que genera una pregunta de test, pero esta vez con el PydanticOutputParser

In [53]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Definimos el modelo Pydantic para la respuesta
class PreguntaTest(BaseModel):
    pregunta: str = Field(description="Texto de la pregunta generada.")
    opciones: List[str] = Field(description="Lista de tres opciones de respuesta.")
    respuesta_correcta: int = Field(description="Índice de la opción correcta (0, 1 o 2).")

# Crear el PydanticOutputParser
output_parser = PydanticOutputParser(pydantic_object=PreguntaTest)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

# Crear el prompt
plantilla = """
Genera una pregunta de test sobre el tema: {tema}.
El nivel de dificultad debe ser: {nivel}.
La pregunta debe tener tres opciones de respuesta y una respuesta correcta.

{format_instructions}
"""

prompt_template = PromptTemplate(
    template=plantilla,
    input_variables=["tema", "nivel"],
    partial_variables={"format_instructions": format_instructions}
)

# Generar el prompt para el LLM
prompt = prompt_template.format(tema="LangChain", nivel="medio")
print("Prompt generado:\n", prompt)

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)

# Hacemos la llamada
respuesta = modelo.invoke(prompt).content

# Parseamos la respuesta fon el parser de salida
respuesta_formateada = output_parser.parse(respuesta)

rprint(respuesta)
rprint(type(respuesta))

rprint(respuesta_formateada)
rprint(type(respuesta_formateada))

# Usamos los datos estructurados
rprint(f"Pregunta: {respuesta_formateada.pregunta}")
rprint("Opciones:")
# La función enumerate toma un iterable (como una lista) y devuelve un objeto que genera pares de valores
# start=1 , para que el 0 se considere 1
for i, opcion in enumerate(respuesta_formateada.opciones):
    rprint(f"{i+1}. {opcion}")

rprint(f"Respuesta correcta: Opción {respuesta_formateada.respuesta_correcta + 1}")


Prompt generado:
 
Genera una pregunta de test sobre el tema: LangChain.
El nivel de dificultad debe ser: medio.
La pregunta debe tener tres opciones de respuesta y una respuesta correcta.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"pregunta": {"description": "Texto de la pregunta generada.", "title": "Pregunta", "type": "string"}, "opciones": {"description": "Lista de tres opciones de respuesta.", "items": {"type": "string"}, "title": "Opciones", "type": "array"}, "respuesta_correcta": {"description": "Índice de la opción correcta (0, 1 o 2).", "title": "Respuesta Cor

```json
{
  "pregunta": "¿Cuál es el propósito principal de LangChain?",
  "opciones": [
    "Proporcionar una plataforma para el desarrollo de aplicaciones de inteligencia artificial",
    "Crear un framework para el análisis de datos y la visualización",
    "Ofrecer una herramienta para la automatización de tareas de procesamiento de lenguaje natural"
  ],
  "respuesta_correcta": 0
}
```

<class 'str'>

PreguntaTest(
    pregunta='¿Cuál es el propósito principal de LangChain?',
    opciones=[
        'Proporcionar una plataforma para el desarrollo de aplicaciones de inteligencia artificial',
        'Crear un framework para el análisis de datos y la visualización',
        'Ofrecer una herramienta para la automatización de tareas de procesamiento de lenguaje natural'
    ],
    respuesta_correcta=0
)

<class '__main__.PreguntaTest'>

...

Opciones:

1. Proporcionar una plataforma para el desarrollo de aplicaciones de inteligencia artificial

2. Crear un framework para el análisis de datos y la visualización

3. Ofrecer una herramienta para la automatización de tareas de procesamiento de lenguaje natural

Respuesta correcta: Opción 1

## **🧪 Ejemplo: Informe médico.**
Veamos otro ejemplo con Pydantic. Dado un informe medico de un paciente, deseamos extraer de forma estructurada la informacion que contiene




In [65]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import datetime

# Modelo Pydantic para representar un informe médico estructurado
class MedicalReport(BaseModel):
    id_paciente: str = Field(description="Identificador único del paciente")
    diagnosticos: List[str] = Field(description="Lista de diagnósticos principales")
    signos_vitales: dict = Field(description="Signos vitales como temperatura, presión arterial, etc.")
    medicaciones: List[dict] = Field(description="Lista de medicamentos con nombre, dosis y frecuencia")
    recomendaciones: List[str] = Field(description="Recomendaciones médicas para el paciente")
    fecha_seguimiento: Optional[str] = Field(description="Fecha recomendada para seguimiento (formato YYYY-MM-DD)")

# Crear el parser
parser = PydanticOutputParser(pydantic_object=MedicalReport)

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0)

# Texto del informe médico no estructurado
informe_medico = """
Paciente: P12345
Fecha de consulta: 15/03/2025
Motivo de consulta: Dolor abdominal y fiebre de 3 días de evolución.
Examen físico: Temperatura 38.2°C, Presión arterial 130/85, Frecuencia cardíaca 95 lpm, Saturación O2 98%.
Abdomen doloroso a la palpación en cuadrante inferior derecho.
Diagnóstico: Apendicitis aguda. Deshidratación leve.
Tratamiento: Ceftriaxona 1g IV cada 12 horas, Metronidazol 500mg IV cada 8 horas,
Paracetamol 1g VO cada 8 horas si fiebre o dolor.
Plan: Programar cirugía. Hidratación intravenosa. Control de signos vitales.
Recomendaciones: Dieta líquida, reposo absoluto, vigilar cambios en el dolor o aparición de fiebre.
Próxima cita: 10 de abril de 2025.
"""

# Crear el prompt
prompt = f"""
Extrae y estructura la información del siguiente informe médico:

{informe_medico}

{parser.get_format_instructions()}
"""

# Obtener la respuesta
respuesta = modelo.invoke(prompt)
# Parsear la respuesta (esto devuelve una instancia de MedicalReport)
informe_estructurado = parser.parse(respuesta.content)

rprint(respuesta.content)
rprint(type(respuesta.content))

rprint(informe_estructurado)
rprint(type(informe_estructurado))



# Usar la instancia
print(f"ID del paciente: {informe_estructurado.id_paciente}")
print(f"Diagnósticos: {', '.join(informe_estructurado.diagnosticos)}")
print("Signos vitales:")
for signo, valor in informe_estructurado.signos_vitales.items():
    print(f"  - {signo}: {valor}")
print("Medicaciones:")
for med in informe_estructurado.medicaciones:
    # Access keys using the names provided by the LLM: 'name', 'dose', 'frequency'
    print(f"  - {med['nombre']} {med['dosis']} {med['frecuencia']}")

print(f"Fecha de seguimiento: {informe_estructurado.fecha_seguimiento}")

A continuación, se presenta la información estructurada en formato JSON según el esquema proporcionado:


```json
{
  "id_paciente": "P12345",
  "diagnosticos": [
    "Apendicitis aguda",
    "Deshidratación leve"
  ],
  "signos_vitales": {
    "temperatura": 38.2,
    "presion_arterial": "130/85",
    "frecuencia_cardiaca": 95,
    "saturacion_o2": 98
  },
  "medicaciones": [
    {
      "nombre": "Ceftriaxona",
      "dosis": "1g",
      "frecuencia": "cada 12 horas",
      "via": "IV"
    },
    {
      "nombre": "Metronidazol",
      "dosis": "500mg",
      "frecuencia": "cada 8 horas",
      "via": "IV"
    },
    {
      "nombre": "Paracetamol",
      "dosis": "1g",
      "frecuencia": "cada 8 horas si fiebre o dolor",
      "via": "VO"
    }
  ],
  "recomendaciones": [
    "Dieta líquida",
    "Reposo absoluto",
    "Vigilar cambios en el dolor o aparición de fiebre"
  ],
  "fecha_seguimiento": "2025-04-10"
}
```

<class 'str'>

MedicalReport(
    id_paciente='P12345',
    diagnosticos=['Apendicitis aguda', 'Deshidratación leve'],
    signos_vitales={
        'temperatura': 38.2,
        'presion_arterial': '130/85',
        'frecuencia_cardiaca': 95,
        'saturacion_o2': 98
    },
    medicaciones=[
        {'nombre': 'Ceftriaxona', 'dosis': '1g', 'frecuencia': 'cada 12 horas', 'via': 'IV'},
        {'nombre': 'Metronidazol', 'dosis': '500mg', 'frecuencia': 'cada 8 horas', 'via': 'IV'},
        {'nombre': 'Paracetamol', 'dosis': '1g', 'frecuencia': 'cada 8 horas si fiebre o dolor', 'via': 'VO'}
    ],
    recomendaciones=['Dieta líquida', 'Reposo absoluto', 'Vigilar cambios en el dolor o aparición de fiebre'],
    fecha_seguimiento='2025-04-10'
)

<class '__main__.MedicalReport'>

ID del paciente: P12345
Diagnósticos: Apendicitis aguda, Deshidratación leve
Signos vitales:
  - temperatura: 38.2
  - presion_arterial: 130/85
  - frecuencia_cardiaca: 95
  - saturacion_o2: 98
Medicaciones:
  - Ceftriaxona 1g cada 12 horas
  - Metronidazol 500mg cada 8 horas
  - Paracetamol 1g cada 8 horas si fiebre o dolor
Fecha de seguimiento: 2025-04-10


Observa que la clase del objeto devuelto NO es un diccionario, ni un JSON sino un objeto heredado de la clase que hemos definido con Pydantic !!


# **12. with_structured_output()**

Vamos a realizar algunos de los ejemplos anteriores, pero esta vez usando las capacidades modernas de los modelos para producir de forma nativa una salida estructurada, con lo cual podemos dejar de usar OutputParsers

## 🧪 Ejemplo: Bibliografia de un autor

Este caso los resolvimos anteriormente con JSONOuputParser y tambien con PydanticOutputParser.

Necesitamos de Pydantic todavia para validaciones e instruir al modelo de una forma mucho mas precisa que si tuviaremos que hacer toda la descripcion de campos del diccionario esparado en un prompt

In [68]:
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI

# Definir la estructura para un libro
class Libro(BaseModel):
    titulo: str = Field(description="Título del libro")
    año: int = Field(description="Año de publicación")

# Definir la estructura para la bibliografía completa
class Bibliografia(BaseModel):
    bibliografia: List[Libro] = Field(description="Lista de libros publicados por el autor")

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0.7)

# Configuramos el modelo para que devuelva directamente una instancia de Bibliografia
modelo_struct = modelo.with_structured_output(Bibliografia)

# Ejemplo de uso
autor = "Gabriel García Márquez"
prompt = f"""
Proporciona ÚNICAMENTE la bibliografía del autor: {autor}.
INSTRUCCIONES ESTRICTAS:
1. Solo crea una lista de sus libros con título y año.
2. La lista de libros debe usar la clave "bibliografia".
3. Cada libro debe tener SOLO los campos "titulo" y "año".
"""

# Hacemos la llamada directamente al modelo estructurado
resultado = modelo_struct.invoke(prompt)


# Imprimir los resultados
print(f"Autor: {autor}")
print("Bibliografía:")
for libro in resultado.bibliografia:
    print(f"Año: {libro.año}, Título: {libro.titulo}")



Autor: Gabriel García Márquez
Bibliografía:
Año: 1955, Título: La hojarasca
Año: 1961, Título: El coronel no tiene quien le escriba
Año: 1962, Título: Los funerales de la Mamá Grande
Año: 1967, Título: Cien años de soledad
Año: 1970, Título: Relato de un náufrago
Año: 1970, Título: El ahogado más hermoso del mundo
Año: 1981, Título: Crónica de una muerte anunciada
Año: 1985, Título: El amor en los tiempos del cólera
Año: 1992, Título: Doce cuentos peregrinos
Año: 1996, Título: Noticia de un secuestro
Año: 2002, Título: Vivir para contarla


## **🧪 Ejemplo: Creacion de un test**

Este ejemplo lo hemos realizado con tres OutputParsers diferentes.
Ahora con  with_structured_output(). Es realmente facil y fiable.

In [73]:
from typing import List
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

# Definimos un modelo Pydantic en lugar de ResponseSchema
class PreguntaTest(BaseModel):
    pregunta: str = Field(description="Texto de la pregunta generada.")
    opciones: List[str] = Field(description="Lista de tres opciones de respuesta.")
    respuesta_correcta: int = Field(description="Índice de la opción correcta (1, 2 o 3).")

# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0.7)

# Configuramos el modelo para que devuelva directamente una instancia de QuizQuestion
modelo_struct = modelo.with_structured_output(PreguntaTest)

# Ejemplo de uso
tema = "LangChain"
nivel = "medio"

prompt = f"""
    Genera una pregunta de test sobre el tema: {tema}.
    El nivel de dificultad debe ser: {nivel}.
    La pregunta debe tener tres opciones de respuesta y una respuesta correcta.
    """

# Hacemos la llamada directamente al modelo estructurado
resultado = modelo_struct.invoke(prompt)
rprint(resultado)
rprint(type(resultado))

# Mostramos los resultados
# Como la respuesta es un objeto Pydantic (no un dict) usamos la notacuón .
print(f"Pregunta: {resultado.pregunta}")
print("Opciones:")
for i, opcion in enumerate(resultado.opciones, start=1):
    print(f"{i}. {opcion}")
print(f"Respuesta correcta: Opción {resultado.respuesta_correcta}")
print("\n\n")
# Si necesitas acceder como diccionario, lo pasamos a dict con .dump
resultado_dict = resultado.model_dump()
rprint(type(resultado_dict))


PreguntaTest(
    pregunta='¿Cuál es el propósito principal de LangChain?',
    opciones=[
        'Proporcionar una plataforma para el desarrollo de aplicaciones de inteligencia artificial',
        'Ofrecer un marco de trabajo para la creación de modelos de lenguaje',
        'Crear un entorno de prueba para algoritmos de aprendizaje automático'
    ],
    respuesta_correcta=2
)

<class '__main__.PreguntaTest'>

Pregunta: ¿Cuál es el propósito principal de LangChain?
Opciones:
1. Proporcionar una plataforma para el desarrollo de aplicaciones de inteligencia artificial
2. Ofrecer un marco de trabajo para la creación de modelos de lenguaje
3. Crear un entorno de prueba para algoritmos de aprendizaje automático
Respuesta correcta: Opción 2





<class 'dict'>

# **13. Ejercicios**
---

## **👨🏻‍🏫 Ejercicio 1: Extractor de Información de Películas**

**Objetivo**: Crear un sistema que extraiga información estructurada sobre películas a partir de una consulta simple.

**Descripción**: Implementa un programa utilizando LangChain y `StructuredOutputParser` que tome como entrada el nombre de una película y genere una ficha técnica con información relevante como título, director, género, año, puntuación y resumen.

**Requisitos**:

1. Utilizar `ResponseSchema` para definir la estructura de los campos a extraer
2. Implementar un `StructuredOutputParser` para procesar la respuesta del modelo
3. Crear un `PromptTemplate` que incluya las instrucciones de formato
4. Integrar el sistema con un modelo de lenguaje (ChatOpenAI o ChatGroq)
5. Procesar y mostrar la información de manera estructurada

**Entregable**: Un script de Python que, al ejecutarse con el nombre de una película como entrada, genere una ficha técnica estructurada con la información solicitada.

**Nivel**: Medio

**Aplicación práctica**: Este sistema puede ser útil para crear bases de datos de películas, generar fichas técnicas automáticas o construir sistemas de recomendación basados en atributos específicos de las películas.

### Resuelve el ejerciccio en este cuaderno. Una solucion la puedes encontrar en el repositorio de esta serie de cuadernos.
https://github.com/juanfranbrv/curso-langchain

In [79]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

#Creamos un esquema de respuesta (ResponseSchema) para cada campo que queremos extraer:
esquema_respuesta = [
            ResponseSchema(name="titulo", description="Titulo de la película."),
            ResponseSchema(name="director", description="Nombre del director de la película"),
            ResponseSchema(name="genero", description="Genero cinematografico de  la pelicula"),
            ResponseSchema(name="año", description="Año de estreno de la película"),
            ResponseSchema(name="puntuacion", description="Número de estrellas 1,2,3,4,5"),
            ResponseSchema(name="resumen", description="Resumen de la crítica de la película")
                    ]

# Crear el StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(esquema_respuesta)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

# Crear el prompt
plantilla = """
        Genera la siguinte informacion sobre esta {pelicula}:
            Titulo
            Director
            Genero
            Puntuacion

        solo devuelve la informacion solicitada en el formato indicado


        {format_instructions}
        """

# Generar el prompt para el LLM
prompt_template = PromptTemplate(
    template=plantilla,
    input_variables=["pelicula"],
    partial_variables={"format_instructions": format_instructions}
)

pelicula = "El señor de los anillos"
prompt = prompt_template.format(pelicula=pelicula)


# Instanciamos el modelo (comenta el que no desees usar)
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)
modelo = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY,temperature=0.7)

respuesta = modelo.invoke(prompt).content

# Parseamos la respuesta con el parser de salida
respuesta_formateada = output_parser.parse(respuesta)

print(respuesta)
rprint(type(respuesta))

rprint(respuesta_formateada)
rprint(type(respuesta_formateada))



```json
{
	"titulo": "El Señor de los Anillos",
	"director": "Peter Jackson",
	"genero": "Aventura, Fantasía",
	"año": "2001",
	"puntuacion": "5",
	"resumen": "Una adaptación épica de la novela de J.R.R. Tolkien, que sigue a un hobbit llamado Frodo en su peligrosa misión para destruir el Anillo Único y salvar a la Tierra Media de las fuerzas oscuras de Sauron."
}
```


<class 'str'>

{
    'titulo': 'El Señor de los Anillos',
    'director': 'Peter Jackson',
    'genero': 'Aventura, Fantasía',
    'año': '2001',
    'puntuacion': '5',
    'resumen': 'Una adaptación épica de la novela de J.R.R. Tolkien, que sigue a un hobbit llamado Frodo en su 
peligrosa misión para destruir el Anillo Único y salvar a la Tierra Media de las fuerzas oscuras de Sauron.'
}

<class 'dict'>

## **👨🏻‍🏫 Ejercicio: Traductor de Código Python a Pseudocódigo**

**Objetivo**: Crear un sistema que analice código Python y lo convierta automáticamente en pseudocódigo estructurado.

**Descripción**: Implementa un programa utilizando LangChain y el método `with_structured_output()` que tome como entrada un fragmento de código Python y genere una representación en pseudocódigo. El sistema debe identificar las estructuras principales del código (bucles, condicionales, asignaciones, etc.) y organizarlas en un formato de pseudocódigo claro y legible.

**Requisitos**:

1. Utilizar los modelos Pydantic para definir la estructura de salida
2. Implementar un prompt efectivo que instruya al modelo sobre cómo analizar el código
3. Generar pseudocódigo que mantenga la lógica y estructura del código original
4. Identificar y categorizar los elementos estructurales del código
5. Presentar los resultados de manera organizada y legible

**Entregable**: Un script de Python que, al ejecutarse con un código de ejemplo, genere su representación estructurada en pseudocódigo.

**Nivel**: Medio

**Aplicación práctica**: Esta herramienta puede ser útil para documentación de algoritmos, material didáctico, o para facilitar la comprensión de código a personas no familiarizadas con Python.  

### Resuelve el ejercicio en este cuaderno. Una solucion la puedes encontrar en el repositorio de esta serie de cuadernos.
https://github.com/juanfranbrv/curso-langchain

In [ ]:
# Tu código aquí

Retos

Reto 1
StructuredOutputParser (Nivel Medio)
Título: Análisis de reseñas de películas
Descripción:
Usa StructuredOutputParser para extraer datos clave de una crítica de cine generada por un modelo de lenguaje.

Requisitos:

Define un esquema con estos campos:

titulo_pelicula (string).

director (string).

genero (string, ej: "ciencia ficción").

puntaje (string, del 1 al 10).

Genera una crítica ficticia de una película usando un prompt que incluya las instrucciones de formato.

Parsea la respuesta y muestra el resultado como diccionario.

Ejemplo de salida:

python
Copy
{'titulo_pelicula': 'El Último Eclipse', 'director': 'Carlos Méndez', 'genero': 'drama', 'puntaje': '8'}  


## Reto 2: Generador de planes de viaje con with\_structured\_output()

**Nivel: Medio**

### Objetivo

Crear un generador de planes de viaje que proporcione itinerarios estructurados utilizando el método `with_structured_output()`.

### Descripción

Desarrolla una aplicación que permita al usuario especificar un destino, duración del viaje y preferencias, y genere un itinerario detallado y estructurado. El sistema debe proporcionar:

- Resumen del viaje (párrafo)
- Itinerario diario (lista de días)
- Recomendaciones de alojamiento (lista de opciones)
- Presupuesto estimado (rangos por categoría)
- Consejos prácticos (lista de recomendaciones)

### Requisitos

1. Define un modelo Pydantic para la estructura del plan de viaje
2. Utiliza `with_structured_output()` para configurar el LLM
3. Crea un prompt claro y conciso (sin necesidad de instrucciones de formato)
4. Implementa una función que genere planes para diferentes destinos
5. Presenta el itinerario de manera visualmente atractiva

### Ejemplo de entrada

Copiar

`Destino: Barcelona Duración: 3 días Preferencias: Arte, gastronomía, presupuesto medio`

### Sugerencia de implementación

Comienza definiendo las clases Pydantic para actividades diarias y el plan completo. Luego configura el modelo con `with_structured_output()` y crea una función principal que procese la solicitud del usuario.

### Extensión opcional

Añade una función para exportar el itinerario a formato Markdown o HTML para compartirlo fácilmente.

Reto 3
Crear un ejemplo que dada una receta la presente estructurada en ingrdientes, pasos, etc
Esta en este video https://www.youtube.com/watch?v=lbWxastyWPw



Referencias:

1. https://freedium.cfd/https://python.plainenglish.io/langchain-in-chains-7-output-parsers-e1a2cdd40cd3

2. https://bobrupakroy.medium.com/harness-llm-output-parsers-for-a-structured-ai-7b456d231834

3. https://cobusgreyling.medium.com/langchain-structured-output-parser-using-openai-c3fe6927beb7

4. https://python.langchain.com/docs/how_to/output_parser_structured/

5. https://www.comet.com/site/blog/mastering-output-parsing-in-langchain/

6. https://www.gettingstarted.ai/how-to-langchain-output-parsers-convert-text-to-objects/

7. https://www.gettingstarted.ai/how-to-extract-metadata-from-pdf-convert-to-json-langchain/  Este es un buen reto

8. https://www.analyticsvidhya.com/blog/2024/11/output-parsers/
